# Import and download the required libraries
For this project we use
* pynrrd for reading our data (we work with .nrrd files)
* numpy for mathematical operations and saving our data
* matplotlib for data visualisation
* ipywidgets for interactive sliders

In [ ]:
!pip install pynrrd

In [ ]:
%matplotlib inline

# Importing libraries needed for data processing
import os
import numpy as np
import nrrd
import pandas as pd
import random
import tensorflow as tf
from numpy import asarray
from numpy import savez_compressed
from numpy import load



# Necessary libraries for data visualisation and animations
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
from scipy import ndimage
from skimage import measure
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection


# Image alteration
import cv2
import scipy
import skimage.io as io
import skimage.transform as trans
from keras import utils

# Time
from tqdm.notebook import tqdm
import time
import datetime


# Loading the data

In [ ]:
def load_data_training():
    """
    Loading the data for training.
    For training we will utilise 23 out of 33
    """
 
    patients=[]
    labels_perete=[]
    labels_tumoare=[]
    
    pacients_names=['baboian-prostata','cristea-prostata','curea','dan ovidiu','georgescu','gherghel','grigoriu',
               'heghes','merca','mocan','moldovan','muresan','patras','patrascu','petrean adrian','pop anton','pop ioan','sabo','saigo',
               'stan','stanca','toader','valean']
    
    for i in range(len(pacients_names)):
        path1='../input/mri-prostate/MRI/'
        path2='../input/mri-prostate/Wall_MRI/'
        path3='../input/mri-prostate/Tumor_MRI/'


        patient, _= nrrd.read(path1+pacients_names[i]+'/'+os.listdir(path1+pacients_names[i])[0])
        label_perete, _= nrrd.read(path2+pacients_names[i]+'/'+os.listdir(path2+pacients_names[i])[0])
        label_tumoare, _= nrrd.read(path3+pacients_names[i]+'/'+os.listdir(path3+pacients_names[i])[0])
        patients.append(patient)
        labels_perete.append(label_perete)
        labels_tumoare.append(label_tumoare)
        
    return patients,labels_perete,labels_tumoare



In [ ]:
def load_data_testing():
    """
    Loading the testing data.
    For testing we will use the remaining 10 out of 33 patients
    """
    patients=[]
    labels_perete=[]
    labels_tumoare=[]
    
    pacients_names=['cazan','cheregi','chesa','coldea','dambar','farcas','lazar','onicau','tripon','zehan']
    
    for i in range(10):
         
        path1='../input/prostatatura2/Date2/'
        path2='../input/prostatatura2/Date2Prostata/'
        path3='../input/prostatatura2/Date2Tumora/'

        patient, _= nrrd.read(path1+pacients_names[i]+'/'+os.listdir(path1+pacients_names[i])[0])
        label_perete, _= nrrd.read(path2+pacients_names[i]+'/'+os.listdir(path2+pacients_names[i])[0])
        label_tumoare, _= nrrd.read(path3+pacients_names[i]+'/'+os.listdir(path3+pacients_names[i])[0])
        patients.append(patient)
        labels_perete.append(label_perete)
        labels_tumoare.append(label_tumoare)
        
    return patients,labels_perete,labels_tumoare



In [ ]:
patients,labels_perete,labels_tumori=load_data_training()
patients2,labels_perete2,labels_tumori2=load_data_testing()

Adding the data from the first and second batch together

In [ ]:
patients+=patients2
labels_perete+=labels_perete2
labels_tumori+=labels_tumori2

Deleting the uneeded data because it occupies space

In [ ]:
del patients2
del labels_perete2
del labels_tumori2

Extrating the data from one patient because they have additional unneeded data

In [ ]:
for i in range(len(patients)):
    if len(patients[i].shape)==4:
        patients[i]=patients[i][0]

In [ ]:
print('We have ' + str(len(patients))+ ' patients')

Showing the shapes of the data

In [ ]:
print('   Patients    '+'|'+'     Wall       '+'|'+'    Tumor')
print('--------------------------------------------')
for i in range(len(patients)):
    print(str(patients[i].shape)+' |'+ str(labels_perete[i].shape) +' |'+str(labels_tumori[i].shape))        

In [ ]:
def show_original_and_wall_and_tumor_figure_1(patient_nr,patient_nr_2,patient_nr_3,x):
    """
    :param x:(int) Original image
    :param nr_pacient:(int) current patient
    
    Showing the original image with both the bladder wall and tumor segmentation
    """
    fig, ax = plt.subplots(3,3, figsize=[12, 12])
    img=patients[patient_nr][:,:,x]
    img_2=labels_perete[patient_nr][:,:,x]
    img_3=labels_tumori[patient_nr][:,:,x]
    img_row2_1=patients[patient_nr_2][:,:,x]
    img_row2_2=labels_perete[patient_nr_2][:,:,x]
    img_row2_3=labels_tumori[patient_nr_2][:,:,x]
    img_row3_1 =patients[patient_nr_3][:,:,x]
    img_row3_2 =labels_perete[patient_nr_3][:,:,x]
    img_row3_3 =labels_tumori[patient_nr_3][:,:,x]
                                           
    rotated_img = ndimage.rotate(img, 270)
    rotated_img_2 = ndimage.rotate(img_2, 270)
    rotated_img_3 = ndimage.rotate(img_3, 270)
    rotated_row2_img = ndimage.rotate(img_row2_1, 270)
    rotated_row2_img_2 = ndimage.rotate(img_row2_2, 270)
    rotated_row2_img_3 = ndimage.rotate(img_row2_3, 270)
    rotated_row3_img = ndimage.rotate(img_row3_1, 270)
    rotated_row3_img_2 = ndimage.rotate(img_row3_2, 270)
    rotated_row3_img_3 = ndimage.rotate(img_row3_3, 270)
    ax[0][0].imshow(rotated_img,cmap='gray')
    ax[0][0].axis('off')
    ax[0][1].imshow(rotated_img_2,cmap='gray')
    ax[0][1].axis('off')
    ax[0][2].imshow(rotated_img_3,cmap='gray')
    ax[0][2].axis('off')
    ax[1][0].imshow(rotated_row2_img,cmap='gray')
    ax[1][0].axis('off')
    ax[1][1].imshow(rotated_row2_img_2,cmap='gray')
    ax[1][1].axis('off')
    ax[1][2].imshow(rotated_row2_img_3,cmap='gray')
    ax[1][2].axis('off')
    ax[2][0].imshow(rotated_row3_img,cmap='gray')
    ax[2][0].axis('off')
    ax[2][1].imshow(rotated_row3_img_2,cmap='gray')
    ax[2][1].axis('off')
    ax[2][2].imshow(rotated_row3_img_3,cmap='gray')
    ax[2][2].axis('off')
    plt.show()

In [ ]:
def show_original_and_wall_and_tumor(x,patient_nr=5):
    """
    :param x:(int) Original image
    :param nr_pacient:(int) current patient
    
    Showing the original image with both the bladder wall and tumor segmentation
    """
    fig, ax = plt.subplots(1,3, figsize=[12, 12])
    img=patients[patient_nr][:,:,x]
    img_2=labels_perete[patient_nr][:,:,x]
    img_3=labels_tumori[patient_nr][:,:,x]
    rotated_img = ndimage.rotate(img, 270)
    rotated_img_2 = ndimage.rotate(img_2, 270)
    rotated_img_3 = ndimage.rotate(img_3, 270)

    ax[0].set_title("Original")
    ax[0].imshow(rotated_img,cmap='gray')
    ax[0].axis('off')
    ax[1].set_title("Bladder Wall")
    ax[1].imshow(rotated_img_2,cmap='gray')
    ax[1].axis('off')
    ax[2].set_title("Tumor")
    ax[2].imshow(rotated_img_3,cmap='gray')
    ax[2].axis('off')
    plt.show()

In [ ]:
def split(X,Y1,Y2,nr):
    """
    :param X:(list of 3D Numpy Arrays) list with the patients of the data
    :param Y:(list of 3D Numpy Arrays) list with the masks
    :returns: The data split in training and testing data
    """
    indexes=[i for i in range(len(X))]
    sample=np.random.choice(indexes,nr,replace=False)
    print("Date chosen for testing")
    print(sample)
    x_train_1 = [X[sample[i]] for i in range(len(sample)) if i<14]
    x_train_2 = [X[sample[i]] for i in range(len(sample)) if i>=14]
    x_test = [X[i] for i in indexes if i not in sample]
    
    print("For the first batch")
    s1 = [sample[i] for i in range(len(sample)) if i<14]
    print(s1)
    print("For the second batch")
    s2 = [sample[i] for i in range(len(sample)) if i>=14]
    print(s2)
    
    d_sample = [i for i in indexes if i not in sample]
    print("Data chosen for testing")
    print(d_sample)
    
    y_train_tum_1 = [Y1[sample[i]] for i in range(len(sample)) if i<14]
    y_train_wall_1 = [Y2[sample[i]] for i in range(len(sample)) if i<14]
    y_train_tum_2 = [Y1[sample[i]] for i in range(len(sample)) if i>=14]
    y_train_wall_2 = [Y2[sample[i]] for i in range(len(sample)) if i>=14]
    y_test_tum = [Y1[i] for i in indexes if i not in sample]
    y_test_wall = [Y2[i] for i in indexes if i not in sample]
    
    
    return x_train_1, x_train_2, x_test, y_train_tum_1, y_train_wall_1, y_train_tum_2, y_train_wall_2, y_test_tum, y_test_wall

In [ ]:
# For the first batch of data
index_1=[6, 26, 8, 5, 14, 21, 18, 29, 0, 32, 12, 1, 20, 11]
# For the second batch of data
index_2=[23, 28, 19, 3, 4, 2, 13, 30, 24, 25, 9, 22, 31, 17]
# Data chosen for testing
index_test=[7, 10, 15, 16, 27]

In [ ]:
def get_patients_from_indexes(X,Y_perete,Y_tumoare,indexes):
    """
    Returns pacients and their respective masks based on the indexes given
    """
    x_train=[X[i] for i in indexes]
    y_train_perete=[Y_perete[i] for i in indexes]
    y_train_tumoare=[Y_tumoare[i] for i in indexes] 
    
    return x_train,y_train_perete,y_train_tumoare

We chose to work with the second batch of data

In [ ]:
patients,labels_perete,labels_tumori = get_patients_from_indexes(patients,labels_perete,labels_tumori,index_2)

In [ ]:
interactive(show_original_and_wall_and_tumor,x=(0,25))

In [ ]:
print('We have ' + str(len(patients))+ ' patients')

# We manually extract the ROI
By using the combined ground truth we extract the region in which both the prostate and the tumour occupy most space.

In [ ]:
def combine_labels(labels_perete,labels_tumori):
    """
    Auxiliary function that helps us combine the labels for extracting the ROI
    :param labels_perete:(list of 3D numpy arrays) list of wall masks
    :param labels_tumori:(list of 3D numpy arrays) list of tumor masks
    
    :returns:
    labels: A list which has, for every 2D map:
    -1 if there is a wall or tumor present in the array
    -0 otherwise
    
    """
    for i in range(len(labels_perete)):
        x=labels_perete[i]
        labels_perete[i]=np.where(x!=0, 255, x)

    for i in range(len(labels_tumori)):
        x=labels_tumori[i]
        labels_tumori[i]=np.where(x!=0, 125, x)

    labels=[]
    
    for i in range(len(labels_perete)):
        labels.append(labels_tumori[i]+labels_perete[i])

    for i in range(len(labels)):
        x=labels[i]
        labels[i]=np.where(x!=0 ,1, x)
        
        
    return labels

List of combined masks that we will take into account when extracting the ROI

In [ ]:
labels=combine_labels(labels_perete,labels_tumori)

In [ ]:
def take_sub_volumes(image_data,label_data,labels_perete,labels_tumoare,sub_x=400,sub_y=400,sub_z=25,max_tries=200,threshold=0.95):
    """
    Takes the ROI manually based on the placement of the wall and tumor
    
    :param image_data:(3D Numpy Array) list of patients
    :param label_data:(3D Numpy Array) - list of obtained masks by combining both the wall and tumour masks
    :param labels_perete:(3D Numpy Array) - list of masks for prostate 
    :param labels_tumoare:(3D Numpy Array) - list of masks for tumour
    :param sub_x:(int) the 1st dimension of the returned sub-volume
    :param sub_y:(int) the 2nd dimension of the returned sub-volume
    :param sub_z:(int) the 3rd dimension of the returned sub-volume
    :param max_tries:(int) - how many times to search for the best sub-volume
    
    :returns:
    X: list of sub-volumes of patients
    Y: list of sub-volumes of masks
    
    """
    p_bar = tqdm(range(max_tries))
    X=None
    Y=None
    forbidden=[]
    reset=0
    if image_data.shape[0]<label_data.shape[0]:
         main_x=image_data.shape[0]
    else:
         main_x=label_data.shape[0]

    if image_data.shape[1]<label_data.shape[1]:
         main_y=image_data.shape[1]
    else:
         main_y=label_data.shape[1]

    if image_data.shape[2]<label_data.shape[2]:
         main_z=image_data.shape[2]
    else:
         main_z=label_data.shape[2]
    
    best_backgr_ratio=0.99
    neighbour_tries=0
     
    best_x=0
    best_y=0
    best_z=0
    
    for tries in p_bar:
        
        p_bar.set_description(f'Working on "{tries}"')
        if main_x==sub_x:
            x=0
        else:
            x=np.random.randint(50,150)
        if main_y==sub_y:
            y=0
        else:
            y=np.random.randint(50,150)
        if main_z==sub_z:
            z=0
        else:
            if main_z-sub_z+1>5:
                z=np.random.randint(5,main_z-sub_z+1)
            else:
                z=0


        sub_volume=label_data[x:x+sub_x,y:y+sub_y,z:z+sub_z]  # we take the sub-volume
        
        
        for volume in forbidden:
            if (sub_volume.shape[0]==volume.shape[0] and sub_volume.shape[1]==volume.shape[1] and sub_volume.shape[2]==volume.shape[2]):
                if ((sub_volume==volume).all()):
                    reset=1
                    if tries==max_tries-1:
                        reset=0
                    forbidden.append(sub_volume)
                    break
    
                
        if (reset==1):
            reset=0
            time.sleep(0.001)
            continue   # repeat because the found sub-volume isn't valid
        
        
        # Calculate the final sub-volume
        background_ratio= np.count_nonzero(label_data[x:x+sub_x,y:y+sub_y,z:z+sub_z]==0) / ((sub_x*sub_y*sub_z)+(1e-7))
        
    
        if background_ratio>0.99 and tries!=max_tries-1:
            forbidden.append(sub_volume)
            time.sleep(0.001)
            continue

        
        
        if background_ratio<best_backgr_ratio:
            best_backgr_ratio=background_ratio
            best_x=x
            best_y=y
            best_z=z
      
        if tries==max_tries-1:
#             p_bar.next()
            Y_perete=labels_perete[best_x:best_x+sub_x,best_y:best_y+sub_y,best_z:best_z+sub_z]
            Y_tumoare=labels_tumoare[best_x:best_x+sub_x,best_y:best_y+sub_y,best_z:best_z+sub_z]
            X=np.copy(image_data[best_x:best_x+sub_x,best_y:best_y+sub_y,best_z:best_z+sub_z]) 
           
            return X,Y_perete,Y_tumoare
        
        time.sleep(0.001)


def resize_volume_take_sub_volume(volume_list,label_list,labels_perete,labels_tumori):
    """
    Resizes the array of 3D Numpy Arrays
    :param volume_list:(Numpy array de 3D Numpy arrays)
    :param label_list:(Numpy array de 3D Numpy arrays) 
    :param labels_perete:(Numpy array de 3D Numpy arrays) 
    :param labels_tumori:(Numpy array de 3D Numpy arrays) 
    :returns:
    :volume_list: The array with the elements resized
    :labels_perete: The array with the bladder wall masks of resized elements
    :labels_tumori: The array with the tumor masks of resized elements
      """
    
    for i in range(len(volume_list)):
        volume_list[i],labels_perete[i],labels_tumori[i]=take_sub_volumes(volume_list[i],label_list[i],labels_perete[i],labels_tumori[i],sub_z=volume_list[i].shape[2])

    return volume_list,labels_perete,labels_tumori

In [ ]:
patients,labels_perete,labels_tumori = resize_volume_take_sub_volume(patients,labels,labels_perete,labels_tumori)

Preprocessing the labels so that for bladder wall the value is 255 and for the tumor 125

In [ ]:
for i in range(len(labels_perete)):
    x=labels_perete[i]
    labels_perete[i]=np.where(x==255, 1, x)

for i in range(len(labels_tumori)):
    x=labels_tumori[i]
    labels_tumori[i]=np.where(x==125, 1, x)

# Data Augmentation
We augment the data using:
* rotation
* flipping
* elastic transformations

In [ ]:
def elastic_transform(image,label_p,label_t, alpha, sigma, alpha_affine, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_ (with modifications).
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
         Convolutional Neural Networks applied to Visual Document Analysis", in
         Proc. of the International Conference on Document Analysis and
         Recognition, 2003.

     Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]
    
    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    image_2 = cv2.warpAffine(label_p, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    image_3 = cv2.warpAffine(label_t, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    
    blur_size = int(4 * sigma) | 1
    dx = cv2.GaussianBlur(
        (random_state.rand(*shape) * 2 - 1).astype(np.float32),
        ksize=(blur_size, blur_size), sigmaX=sigma) * alpha
    dy = cv2.GaussianBlur(
        (random_state.rand(*shape) * 2 - 1).astype(np.float32),
        ksize=(blur_size, blur_size), sigmaX=sigma) * alpha
    dz = np.zeros_like(dx)

    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    return (ndimage.map_coordinates(image, indices, order=1, mode='reflect').reshape(shape),
ndimage.map_coordinates(image_2, indices, order=1, mode='reflect').reshape(shape),
ndimage.map_coordinates(image_3, indices, order=1, mode='reflect').reshape(shape))

In [ ]:
# Function used for image transformation

def image_rotate3D_given_angle(img_numpy, angle):
    """
    Rotates a 3D Numpy array with the given angle
    :param img_numpy: 3D numpy array
    :param min_angle:(int) angle in grades
    :param max_angle:(int) angle in grades
    :returns: 2 3D Numpy arrays
    """
    assert img_numpy.ndim == 3, "The array needs to be 3D"
    assert angle > -360 or angle < 360 ,"The angle needs to be between -360 and 360"
    
    return scipy.ndimage.rotate(img_numpy, angle,order=1,mode='constant')

# Function used for mask alteration

def masks_rotated3D_given_angle(mask_volume,angle):
    """
    Individual function for rotating masks
    :param mask_volume: (3D Numpy array) the mask
    :param angle:(int) the angle for rotating
    """
    mask_r=image_rotate3D_given_angle(mask_volume,angle)
    mask_r=utils.to_categorical(mask_r,0)
    mask_r=mask_r[:,:,:,1]

    return mask_r

# Flipping the volume
def flip(data):
    """
    Horizontal flipping a 3D Numpy array
    :param data: (3D numpy array)
    :returns: a 3D Numpy array flipped
    """
    axis=0
    volume=data
    
    volume = np.asarray(volume).swapaxes(axis, 0)
    volume = volume[::-1, ...]
    volume = volume.swapaxes(0, axis)
    volume = np.squeeze(volume)

    return volume

Below there are several functions for augmenting the full list of patients and their respective masks

In [ ]:
def Augment_patient_2(patient,patient_mask_perete,patient_mask_tumoare):
    """
    It adds 8 transformed images of the given patient and their respective transformed masks
    returns it as a list,
  
    :param patient:(3D Numpy Array) Pacient that will be transformed
    :param patient_mask_perete:(3D Numpy Array) Their wall mask
    :param patient_mask_tumoare:(3D Numpy Array) Their tumor mask
    """
    X=[patient]
    Y_perete=[patient_mask_perete]
    Y_tumoare=[patient_mask_tumoare]
#     If validation data is needed, uncomment all of these below
    X_validare=[]
    Y_validare_p=[]
    Y_validare_t=[]
    
    all_angles=[15,14,13,12,11,10,9,8,-15,-14,-13,-12,-11,10,-9,-8]
    
    positions=np.random.choice(all_angles,size=3)
 

    # Roteste volumul cu unghiul dat
    arr_new_patient=image_rotate3D_given_angle(patient,positions[0])
    arn_new_label_parete=masks_rotated3D_given_angle(patient_mask_perete,positions[0])
    arn_new_label_tumoare=masks_rotated3D_given_angle(patient_mask_tumoare,positions[0])
    X.append(arr_new_patient)
    Y_perete.append(arn_new_label_parete)
    Y_tumoare.append(arn_new_label_tumoare)

    # Rastoarna poza si mastile
    flipped_patient=flip(patient)
    flipped_mask_perete=flip(patient_mask_perete)
    flipped_mask_tumoare=flip(patient_mask_tumoare)
    
    # If validation data is needed, uncomment all of these below

    arr_new_patient=image_rotate3D_given_angle(flipped_patient,positions[1])
    arn_new_label_parete=masks_rotated3D_given_angle(flipped_mask_perete,positions[1])
    arn_new_label_tumoare=masks_rotated3D_given_angle(flipped_mask_tumoare,positions[1])
    X_validare=arr_new_patient
    Y_validare_p=arn_new_label_parete
    Y_validare_t=arn_new_label_tumoare
    
    
    arr_new_patient=image_rotate3D_given_angle(flipped_patient,positions[2])
    arn_new_label_parete=masks_rotated3D_given_angle(flipped_mask_perete,positions[2])
    arn_new_label_tumoare=masks_rotated3D_given_angle(flipped_mask_tumoare,positions[2])
    X.append(arr_new_patient)
    Y_perete.append(arn_new_label_parete)
    Y_tumoare.append(arn_new_label_tumoare)
    
    
    arr_new_patient,arn_new_label_parete,arn_new_label_tumoare=elastic_transform(patient,patient_mask_perete,patient_mask_tumoare, patient.shape[1] * 2, patient.shape[1]* 0.08, patient.shape[1] * 0.08)
    X.append(arr_new_patient)
    Y_perete.append(arn_new_label_parete)
    Y_tumoare.append(arn_new_label_tumoare)
    
    return X,Y_perete,Y_tumoare,X_validare,Y_validare_p,Y_validare_t

In [ ]:
def Augment_patient(patient,patient_mask_perete,patient_mask_tumoare):
    """
    It adds 8 transformed images of the given patient and their respective transformed masks
    returns it as a list,
  
    :param patient:(3D Numpy Array) Pacient that will be transformed
    :param patient_mask_perete:(3D Numpy Array) Their wall mask
    :param patient_mask_tumoare:(3D Numpy Array) Their tumor mask
    """
    X=[patient]
    Y_perete=[patient_mask_perete]
    Y_tumoare=[patient_mask_tumoare]
#     If validation data is needed, uncomment all of these below
    X_validare=[]
    Y_validare_p=[]
    Y_validare_t=[]
    all_angles_neg=[-15,-14,-13,-12,-11,10,-9,-8]
    all_angles_pos=[15,14,13,12,11,10,9,8]
    
    positions_pos=np.random.choice(all_angles_pos,size=1)
    positions_neg=np.random.choice(all_angles_neg,size=1)
    positions=[]
 
    positions.append(positions_pos[0])
    positions.append(positions_neg[0])

    for i in range(0,2):
        # Roteste volumul cu unghiul dat
        arr_new_patient=image_rotate3D_given_angle(patient,positions[i])
        arn_new_label_parete=masks_rotated3D_given_angle(patient_mask_perete,positions[i])
        arn_new_label_tumoare=masks_rotated3D_given_angle(patient_mask_tumoare,positions[i])
        X.append(arr_new_patient)
        Y_perete.append(arn_new_label_parete)
        Y_tumoare.append(arn_new_label_tumoare)

    # Rastoarna poza si mastile
    flipped_patient=flip(patient)
    flipped_mask_perete=flip(patient_mask_perete)
    flipped_mask_tumoare=flip(patient_mask_tumoare)
    
    positions_pos=np.random.choice(all_angles_pos,size=2)
    positions_neg=np.random.choice(all_angles_neg,size=2)
    positions=[]
    
    positions.append(positions_pos[0])
    positions.append(positions_neg[0])
    
    # If validation data is needed, uncomment all of these below

    arr_new_patient=image_rotate3D_given_angle(flipped_patient,positions_pos[1])
    arn_new_label_parete=masks_rotated3D_given_angle(flipped_mask_perete,positions_pos[1])
    arn_new_label_tumoare=masks_rotated3D_given_angle(flipped_mask_tumoare,positions_pos[1])
    X_validare=arr_new_patient
    Y_validare_p=arn_new_label_parete
    Y_validare_t=arn_new_label_tumoare
    
    
    for i in range(0,2):
        arr_new_patient=image_rotate3D_given_angle(flipped_patient,positions[i])
        arn_new_label_parete=masks_rotated3D_given_angle(flipped_mask_perete,positions[i])
        arn_new_label_tumoare=masks_rotated3D_given_angle(flipped_mask_tumoare,positions[i])
        X.append(arr_new_patient)
        Y_perete.append(arn_new_label_parete)
        Y_tumoare.append(arn_new_label_tumoare)
    
    arr_new_patient,arn_new_label_parete,arn_new_label_tumoare=elastic_transform(patient,patient_mask_perete,patient_mask_tumoare, patient.shape[1] * 2, patient.shape[1]* 0.08, patient.shape[1] * 0.08)
    X.append(arr_new_patient)
    Y_perete.append(arn_new_label_parete)
    Y_tumoare.append(arn_new_label_tumoare)
    
    return X,Y_perete,Y_tumoare,X_validare,Y_validare_p,Y_validare_t


def augment_data_each(patients_s,labels_perete_s,labels_tumoare_s):
    """
    For every patients it adds 8 transformed images to our list of pacients,
    and their respective transformed masks to our list of masks
    """
    X=[]
    Y_perete=[]
    Y_tumoare=[]
    
    number_of_samples=len(patients_s)
    # If validation data is needed, uncomment all of these below

    valid=[]
    valid_p=[]
    valid_t=[]
    i=0
    patients_f=[]
    labels_perete_f=[]
    labels_tumoare_f=[]
    
    for i in range(number_of_samples):
        X,Y_perete,Y_tumoare,X_validare,Y_validare_p,Y_validare_t=Augment_patient_2(patients_s[i],labels_perete_s[i],labels_tumoare_s[i])
        patients_f+=X
        labels_perete_f+=Y_perete
        labels_tumoare_f+=Y_tumoare
        # If validation data is needed, uncomment all of these below
        valid.append(X_validare)
        valid_p.append(Y_validare_p)
        valid_t.append(Y_validare_t)


    return patients_f,labels_perete_f,labels_tumoare_f,valid,valid_p,valid_t



In [ ]:
def augment_with_angle(patient,patient_mask_perete,patient_mask_tumoare):
    
    all_angles_pos=[15,14,13,12,11,10,9,8]
    all_angles_neg=[-15,-14,-13,-12,-11,-10,-9,-8]
    
    position=list(np.random.choice(all_angles_pos,size=2))
    position.append(np.random.choice(all_angles_neg,size=1)[0])
#     print(position)
    new_patient=flip(patient)
    new_mask_perete=flip(patient_mask_perete)
    new_mask_tumoare=flip(patient_mask_tumoare)
    arr_new_patient=[]
    arn_new_label_parete=[]
    arn_new_label_tumoare=[]
    for i in range(0,2):
        arr_new_patient.append(image_rotate3D_given_angle(new_patient,position[i]))
        arn_new_label_parete.append(image_rotate3D_given_angle(new_mask_perete,position[i]))
        arn_new_label_tumoare.append(image_rotate3D_given_angle(new_mask_tumoare,position[i]))
        
    arr_new_patient.append(image_rotate3D_given_angle(patient,position[2]))
    arn_new_label_parete.append(image_rotate3D_given_angle(patient_mask_perete,position[2]))
    arn_new_label_tumoare.append(image_rotate3D_given_angle(patient_mask_tumoare,position[2]))
    
    return arr_new_patient,arn_new_label_parete,arn_new_label_tumoare
    

In [ ]:
def augment_with_angle_only_one(patient,patient_mask_perete,patient_mask_tumoare,pos=True,flipped=False):
    
    all_angles_pos=[15,14,13,12,11,10,9,8]
    all_angles_neg=[-15,-14,-13,-12,-11,10,-9,-8]
    
    if pos is True:
        position=np.random.choice(all_angles_pos,size=1)[0]
    else:
        position=np.random.choice(all_angles_neg,size=1)[0]
        
    if flipped is True:
        new_patient=flip(patient)
        new_mask_perete=flip(patient_mask_perete)
        new_mask_tumoare=flip(patient_mask_tumoare)
    else:
        new_patient=patient[:]
        new_mask_perete=patient_mask_perete[:]
        new_mask_tumoare=patient_mask_tumoare[:]
  
    arr_new_patient=image_rotate3D_given_angle(new_patient,position)
    arn_new_label_parete=image_rotate3D_given_angle(new_mask_perete,position)
    arn_new_label_tumoare=image_rotate3D_given_angle(new_mask_tumoare,position)
    
    return arr_new_patient, arn_new_label_parete,arn_new_label_tumoare

In [ ]:
def augment_data_each_caz_2(patients_s,labels_perete_s,labels_tumoare_s):
    """
    For every patients it adds 8 transformed images to our list of pacients,
    and their respective transformed masks to our list of masks
    """
    X=[]
    Y_perete=[]
    Y_tumoare=[]
    
    number_of_samples=len(patients_s)
    patients_f=[]
    labels_perete_f=[]
    labels_tumoare_f=[]
    k=0
    while True:
        k+=1
        for i in range(number_of_samples):
            if len(patients_f)==110:
                return patients_f,labels_perete_f,labels_tumoare_f
            if k==1:
                X,Y_perete,Y_tumoare=augment_with_angle(patients_s[i],labels_perete_s[i],labels_tumoare_s[i])
                patients_f+=X
                labels_perete_f+=Y_perete
                labels_tumoare_f+=Y_tumoare
            elif k==2:
                X,Y_perete,Y_tumoare=augment_with_angle_only_one(patients_s[i],labels_perete_s[i],labels_tumoare_s[i],pos=False,flipped=False)
                patients_f.append(X)
                labels_perete_f.append(Y_perete)
                labels_tumoare_f.append(Y_tumoare)

    return patients_f,labels_perete_f,labels_tumoare_f

In [ ]:
patients,labels_perete,labels_tumori,patients_validare,labels_perete_validare,labels_tumori_validare = augment_data_each(patients,labels_perete,labels_tumori)

Show the list of patients after augmenting

In [ ]:
print('How many patients for training we have after augmenting the data:', len(patients))
print('How many patients for validating we have after augmenting the data:', len(patients_validare))

In [ ]:
interactive(show_original_and_wall_and_tumor,x=(0,25))

In [ ]:
def show_original_and_wall_and_tumor_aug(x,patient_nr=4):
    """
    :param x:(int) Original image
    :param nr_pacient:(int) current patient
    
    Showing the original image with both the bladder wall and tumor segmentation
    """
    fig, ax = plt.subplots(1,3, figsize=[12, 12])
    img=patients[patient_nr][:,:,x]
    img_2=labels_perete[patient_nr][:,:,x]
    img_3=labels_tumori[patient_nr][:,:,x]
    rotated_img = ndimage.rotate(img, 270)
    rotated_img_2 = ndimage.rotate(img_2, 270)
    rotated_img_3 = ndimage.rotate(img_3, 270)

    ax[0].set_title("Original")
    ax[0].imshow(rotated_img)
    ax[0].axis('off')
    ax[1].set_title("Bladder Wall")
    ax[1].imshow(rotated_img_2)
    ax[1].axis('off')
    ax[2].set_title("Tumor")
    ax[2].imshow(rotated_img_3)
    ax[2].axis('off')
    plt.show()

In [ ]:
def show_augmented(x,nr=10,nr_2=10):
    '''
    Shows an example of the modified pacients
    '''
    fig, ax = plt.subplots(2,3, figsize=[10, 10])
    fig.suptitle('Original masks vs transformed', fontsize=16)
    img_label_1=patients[x][:,:,nr]
    img_label_2=labels_perete[x][:,:,nr]
    img_label_3=labels_tumori[x][:,:,nr]
    img_label_4=patients[nr_2][:,:,nr]
    img_label_5=labels_perete[nr_2][:,:,nr]
    img_label_6=labels_tumori[nr_2][:,:,nr]
    img_label_1 = ndimage.rotate(img_label_1, 270)
    img_label_2 = ndimage.rotate(img_label_2, 270)
    img_label_3 = ndimage.rotate(img_label_3, 270)
    img_label_4 = ndimage.rotate(img_label_4, 270)
    img_label_5 = ndimage.rotate(img_label_5, 270)
    img_label_6 = ndimage.rotate(img_label_6, 270)

    ax[0][0].imshow(img_label_1,cmap='gray')
    ax[0][0].axis('off')
    ax[0][1].imshow(img_label_2,cmap='gray')
    ax[0][1].axis('off')
    ax[0][2].imshow(img_label_3,cmap='gray')
    ax[0][2].axis('off')
    ax[1][0].imshow(img_label_4,cmap='gray')
    ax[1][0].axis('off')
    ax[1][1].imshow(img_label_5,cmap='gray')
    ax[1][1].axis('off')
    ax[1][2].imshow(img_label_6,cmap='gray')
    ax[1][2].axis('off')
 
    plt.show()

# Resizing the data
* This is a cosequence of the limited RAM given in Kaggle

In [ ]:
def resize(volumes,labels_perete,labels_tumoare,shape_x,shape_y,shape_z):
    """
    Resizes our pacients to (128,128,32) because of limited RAM space
    :param volumes:(3D Numpy Array) The list of pacients that will be resized
    :param labels:(3D Numpy Array) The resized array
    """

    volume_resized=np.zeros((len(volumes),shape_x,shape_y,shape_z))
    labels_perete_resized=np.zeros((len(volumes),shape_x,shape_y,shape_z))
    labels_tumoare_resized=np.zeros((len(volumes),shape_x,shape_y,shape_z))
 
    for i in range(len(volumes)):
        depth_volume=volumes[i].shape[2]
        for z in range(depth_volume):
            if z>=shape_z:
                break
            img=volumes[i][:,:,z]
            img_resized = cv2.resize(img, (shape_x, shape_y), interpolation = cv2.INTER_NEAREST)
            lbl_perete=labels_perete[i][:,:,z]
            lbl_tumoare=labels_tumoare[i][:,:,z]

            lbl_perete_resized = cv2.resize(lbl_perete, (shape_x, shape_y), interpolation = cv2.INTER_NEAREST)
            lbl_tumoare_resized = cv2.resize(lbl_tumoare, (shape_x, shape_y), interpolation = cv2.INTER_NEAREST)
            
            volume_resized[i][:,:,z]=img_resized
            labels_perete_resized[i][:,:,z]=lbl_perete_resized
            labels_tumoare_resized[i][:,:,z]=lbl_tumoare_resized
            
        if depth_volume<shape_z:
            for j in range(shape_z-depth_volume):
                volume_resized[i][:,:,depth_volume+j]=volume_resized[i][:,:,depth_volume-1]
                labels_perete_resized[i][:,:,depth_volume+j]=labels_perete_resized[i][:,:,depth_volume-1]
                labels_tumoare_resized[i][:,:,depth_volume+j]=labels_tumoare_resized[i][:,:,depth_volume-1]

    return volume_resized,labels_perete_resized,labels_tumoare_resized

In [ ]:
patients,labels_perete,labels_tumori = resize(patients,labels_perete,labels_tumori,128,128,32)
patients_validare,labels_perete_validare,labels_tumori_validare = resize(patients_validare,labels_perete_validare,labels_tumori_validare,128,128,32)

In [ ]:
interactive(show_original_and_wall_and_tumor_aug,x=(0,32))

## Standardize the values

* We have to standardize the values because it will make the performance faster and easier for the model to make calculations


In [ ]:
def standardize(volumes):
    """
    We make each volume have a standard deviation of 1 and a mean of 0
    :param volumes: The list of patients
    :returns: (3D Numpy Array) The list of standardized volumes
    """
    standardized_image = np.zeros(volumes.shape)

    for c in range(volumes.shape[0]):
        for z in range(volumes.shape[3]):
           
            image_slice = volumes[c,:,:,z]
            
            centered = image_slice-np.mean(image_slice)
            
            if np.std(centered) != 0:
                centered_scaled = np.std(centered)
                standardized_image[c, :, :, z] = centered / centered_scaled

    return standardized_image

In [ ]:
def min_max(x):
    x_min = np.amin(x,axis=(1, 2), keepdims=True)
    x_max = np.amax(x,axis=(1, 2), keepdims=True)
    x = (x - x_min)/(x_max-x_min)
    return x
    

We apply the standardization

In [ ]:
patients = standardize(patients)
patients_validare = standardize(patients_validare)

# Combining masks for training
We have to combine the masks so they can be used as input for the model

In [ ]:
def replace_value(data,value_replace,value):
    """
    Replaces value_replace with value in a 3D Numpy Array
    :param data: (3D Numpy array) the data in which we will replace
    :param value_replace:(int)
    :param value:(int)
    :returns:
      data modificata
    """
    for i in range(len(data)):
        x=data[i]
        data[i]=np.where(x==value_replace,value, x)
    return data

We place different values for the prostate and the tumor, for prostate we use 125 and for the tumour 255

In [ ]:
labels_perete = replace_value(labels_perete,1,125)
labels_tumori = replace_value(labels_tumori,1,255)

labels_perete_validare = replace_value(labels_perete_validare,1,125)
labels_tumori_validare = replace_value(labels_tumori_validare,1,255)

In [ ]:
def combine_labels_separate(labels_perete,labels_tumori):
    """
    Combines the masks for the wall and tumor
    """
    for i in range(len(labels_perete)):
        x=labels_perete[i]
        labels_perete[i]=np.where(x!=0, 255, x)

    for i in range(len(labels_tumori)):
        x=labels_tumori[i]
        labels_tumori[i]=np.where(x!=0, 125, x)

    labels=[]
    for i in range(len(labels_perete)):
        labels.append(labels_tumori[i]+labels_perete[i])
    
    return labels

Combining the masks for wall and tumour

In [ ]:
labels=combine_labels_separate(labels_perete,labels_tumori)
labels_validare=combine_labels_separate(labels_perete_validare,labels_tumori_validare)
labels=np.array(labels)
labels_validare=np.array(labels_validare)

After combining there might be overlapping regions so we will replace them with the values for tumor, because we are more interested with those

In [ ]:
labels=replace_value(labels,380,125)
labels_validare=replace_value(labels_validare,380,125)

The values need to be one-hot-encoded so they have to be in this interval [0,num_classes-1]

We replace the values for tumor with 1 and for the wall with 2 for the encoder

In [ ]:
def replacing_valued_for_encoding(labels):
    """
    Auxiliary function to changing values
    """
    for i in range(len(labels)):
        x=labels[i]
        labels[i]=np.where(x==255, 2, x)
        labels[i]=np.where(x==125, 1, x)

    return labels

In [ ]:
labels = replacing_valued_for_encoding(labels)
labels_validare = replacing_valued_for_encoding(labels_validare)

In [ ]:
def original_and_mask(x,nr=5):
    """
    Show the combined mask
    """
    fig, ax = plt.subplots(1,2, figsize=[12, 12])
    img_org=patients[nr][:,:,x]
    img_label=labels[nr][:,:,x]
    rotated_img = ndimage.rotate(img_org, 270)
    rotated_img_lb = ndimage.rotate(img_label, 270)
    ax[0].set_title("Original")
    ax[0].imshow(rotated_img,cmap='gray')
    ax[0].axis('off')
    ax[1].set_title("Segmentation")
    ax[1].imshow(rotated_img_lb,cmap='gray')
    ax[1].axis('off')
    plt.show()

In [ ]:
def original_and_mask_validate(x,nr=2):
    """
    Show the combined mask for validation data
    """
    fig, ax = plt.subplots(1,2, figsize=[12, 12])
    img_org=patients_validare[nr][:,:,x]
    img_label=labels_validare[nr][:,:,x]
    rotated_img = ndimage.rotate(img_org, 270)
    rotated_img_lb = ndimage.rotate(img_label, 270)
    ax[0].set_title("Original")
    ax[0].imshow(rotated_img,cmap='gray')
    ax[0].axis('off')
    ax[1].set_title("Segmentation")
    ax[1].imshow(rotated_img_lb,cmap='gray')
    ax[1].axis('off')
    plt.show()

In [ ]:
interactive(original_and_mask_validate,x=(0,25))

# Prepare data for training
We need to
* expand the dimension of the array so it can be used as input
* assure that the data is a Numpy array

In [ ]:
patients = np.expand_dims(patients,-1)
patients_validare = np.expand_dims(patients_validare,-1)

In [ ]:
x_train=np.array(patients)
y_train=np.array(labels)

x_valid=np.array(patients_validare)
y_valid=np.array(labels_validare)

In [ ]:
y_train = utils.to_categorical(y_train,3)
y_valid = utils.to_categorical(y_valid,3)

# Padding the data
So all the patients have the same shape

In [ ]:
def padding_array_input(arr,ref_shape):
    result = np.zeros(ref_shape)
    result[:arr.shape[0],:arr.shape[1],:arr.shape[2],:] = arr[:,:,:,:]
    for i in range(ref_shape[2]-arr.shape[2]):
        result[:arr.shape[0],:arr.shape[1],arr.shape[2]+i,:]=arr[:,:,-1,:]
    return result

def padding_array_output(arr,ref_shape):
    result = np.zeros(ref_shape)
    result[:arr.shape[0],:arr.shape[1],:arr.shape[2],0] = arr[:,:,:,0]
    result[:arr.shape[0],:arr.shape[1],:arr.shape[2],1] = arr[:,:,:,1]
    result[:arr.shape[0],:arr.shape[1],:arr.shape[2],2] = arr[:,:,:,2]
    for i in range(ref_shape[2]-arr.shape[2]):
        result[:arr.shape[0],:arr.shape[1],arr.shape[2]+i,0] = arr[:,:,-1,0]
        result[:arr.shape[0],:arr.shape[1],arr.shape[2]+i,1] = arr[:,:,-1,1]
        result[:arr.shape[0],:arr.shape[1],arr.shape[2]+i,2] = arr[:,:,-1,2]
    return result

In [ ]:
def patient_and_combined_mask(x,nr=20):
    """
    show the one patients and their respective combined mask
    """
    fig, ax = plt.subplots(1,2, figsize=[12, 12])
    img_org=x_train[nr][:,:,x]
    img_label=y_train[nr][:,:,x]
    rotated_img = ndimage.rotate(img_org, 270)
    rotated_img_lb = ndimage.rotate(img_label, 270)
    ax[0].set_title("Original")
    ax[0].imshow(rotated_img,cmap='gray')
    ax[0].axis('off')
    ax[1].set_title("Segmentation")
    ax[1].imshow(rotated_img_lb,cmap='gray')
    ax[1].axis('off')
    plt.show()

In [ ]:
interactive(patient_and_combined_mask,x=(0,32))

In [ ]:
print('|      Input shape     |     Output shape     |')
print('|----------------------|----------------------|')
print('|'+str(x_train.shape)+'|'+str(y_train.shape)+'|')
print('|'+str(x_valid.shape)+' |'+str(y_valid.shape)+' |')

# Saving the preprocessed data
Saving the data as .npz files using numpy

In [ ]:
savez_compressed('x_train.npz', x_train)
savez_compressed('y_train.npz', y_train)

In [ ]:
savez_compressed('x_valid.npz', x_valid)
savez_compressed('y_valid.npz', y_valid)